In [ ]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("pavansubhasht/ibm-hr-analytics-attrition-dataset")

# print("Path to dataset files:", path)

/Users/maksim/PycharmProjects/pythonProject4/env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 50.1k/50.1k [00:00<00:00, 261kB/s]

Extracting files...


Path to dataset files: /Users/maksim/.cache/kagglehub/datasets/pavansubhasht/ibm-hr-analytics-attrition-dataset/versions/1


In [2]:
import pandas as pd
df = pd.read_csv('WA_Fn-UseC_-HR-Employee-Attrition.csv')

In [3]:
df.head(10)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2
5,32,No,Travel_Frequently,1005,Research & Development,2,2,Life Sciences,1,8,...,3,80,0,8,2,2,7,7,3,6
6,59,No,Travel_Rarely,1324,Research & Development,3,3,Medical,1,10,...,1,80,3,12,3,2,1,0,0,0
7,30,No,Travel_Rarely,1358,Research & Development,24,1,Life Sciences,1,11,...,2,80,1,1,2,3,1,0,0,0
8,38,No,Travel_Frequently,216,Research & Development,23,3,Life Sciences,1,12,...,2,80,0,10,2,3,9,7,1,8
9,36,No,Travel_Rarely,1299,Research & Development,27,3,Medical,1,13,...,2,80,2,17,3,2,7,7,7,7


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EmployeeNumber            1470 non-null   int64 
 10  EnvironmentSatisfaction   1470 non-null   int64 
 11  Gender                    1470 non-null   object
 12  HourlyRate                1470 non-null   int64 
 13  JobInvolvement            1470 non-null   int64 
 14  JobLevel                

In [19]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, FactorAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve
import warnings

warnings.filterwarnings('ignore')

OUT_DIR = Path("hr_coursework_output")
OUT_DIR.mkdir(exist_ok=True)

COLOR_SCHEME = {
    'primary': '#264653',
    'positive': '#2A9D8F',
    'negative': '#E76F51',
    'accent': '#E9C46A',
    'neutral': '#8D99AE',
    'background': '#FFFFFF',
    'grid': '#E5E5E5'
}
PALETTE = [COLOR_SCHEME['positive'], COLOR_SCHEME['negative'], COLOR_SCHEME['accent'], COLOR_SCHEME['primary']]
sns.set_theme(style='whitegrid')
plt.rcParams.update({'font.family': 'sans-serif', 'font.size': 12, 'axes.titlesize': 14})


def save_fig(fig, fname, dpi=200):
    path = OUT_DIR / fname
    fig.savefig(path, dpi=dpi, bbox_inches='tight', facecolor='white')
    plt.close(fig)
    print("Saved:", path)


csv_path = "WA_Fn-UseC_-HR-Employee-Attrition.csv"
df = pd.read_csv(csv_path)
df['Attrition_flag'] = df['Attrition'].map({'Yes': 1, 'No': 0})
print("Loaded:", csv_path, "shape:", df.shape)

JOB_ROLE_TRANSLATIONS = {
    'Sales Executive': 'Руководитель отдела продаж', 'Research Scientist': 'Научный сотрудник',
    'Laboratory Technician': 'Лаборант', 'Manufacturing Director': 'Директор по производству',
    'Healthcare Representative': 'Медицинский представитель', 'Manager': 'Менеджер',
    'Sales Representative': 'Торговый представитель', 'Research Director': 'Научный руководитель',
    'Human Resources': 'Кадровик'
}
if 'JobRole' in df.columns:
    df['JobRole_ru'] = df['JobRole'].map(lambda x: JOB_ROLE_TRANSLATIONS.get(x, x))

sns.set_palette(PALETTE)

fig = plt.figure(figsize=(8, 6))
attr_counts = df['Attrition'].value_counts()
labels = ['Остались в компании' if v == 'No' else 'Уволились' for v in attr_counts.index]
colors = [COLOR_SCHEME['positive'] if v == 'No' else COLOR_SCHEME['negative'] for v in attr_counts.index]
wedges, texts, autotexts = plt.pie(attr_counts.values, labels=labels, autopct='%1.1f%%', startangle=90, colors=colors, explode=[0, 0.06])
for a in autotexts:
    a.set_color('white')
    a.set_fontweight('bold')
plt.title('Распределение сотрудников по статусу текучести', fontsize=16, fontweight='bold')
plt.legend(wedges, [f"{lab}: {cnt} чел." for lab, cnt in zip(labels, attr_counts.values)], loc='center left', bbox_to_anchor=(1, 0.5))
save_fig(fig, '1_attrition_pie.png')

fig = plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='Age', hue='Attrition', bins=20, multiple='layer', shrink=0.8,
             palette={'Yes': COLOR_SCHEME['negative'], 'No': COLOR_SCHEME['positive']})
plt.title('Распределение возраста по статусу текучести', fontsize=14, fontweight='bold')
plt.xlabel('Возраст, лет')
plt.ylabel('Количество сотрудников')
save_fig(fig, '2_age_distribution.png')

df['Attrition_ru'] = df['Attrition'].map({'Yes': 'Уволились', 'No': 'Остались'})
fig = plt.figure(figsize=(10, 6))
sns.boxplot(x='Attrition_ru', y='MonthlyIncome', data=df,
            palette={'Остались': COLOR_SCHEME['positive'], 'Уволились': COLOR_SCHEME['negative']})
plt.title('Распределение месячного дохода по статусу текучести', fontsize=14, fontweight='bold')
plt.xlabel('Статус текучести')
plt.ylabel('Месячный доход')
save_fig(fig, '3_income_boxplot.png')

if 'JobRole_ru' in df.columns:
    job_attr = df.groupby('JobRole_ru')['Attrition_flag'].mean().sort_values(ascending=False)
    fig = plt.figure(figsize=(10, 6))
    sns.barplot(x=job_attr.values, y=job_attr.index, palette='viridis')
    plt.xlabel('Доля текучести')
    plt.title('Доля текучести по должностям', fontsize=14, fontweight='bold')
    plt.ylabel('Должность')
    for i, v in enumerate(job_attr.values):
        plt.text(v + 0.005, i, f"{v:.1%}", va='center')
    save_fig(fig, '4_attrition_by_jobrole.png')

num_cols = [c for c in df.select_dtypes(include=[np.number]).columns if c not in ['EmployeeNumber', 'EmployeeCount', 'StandardHours']]
corr = df[num_cols].corr()
corr.to_csv(OUT_DIR / 'full_correlation_matrix.csv')

threshold = 0.35
pairs = [(a, b, corr.loc[a, b]) for i, a in enumerate(num_cols) for j, b in enumerate(num_cols[i + 1:]) if abs(corr.loc[a, b]) >= threshold]
pairs_df = pd.DataFrame(pairs, columns=['feature_a', 'feature_b', 'r']).sort_values('r', key=lambda s: s.abs(), ascending=False)
pairs_df.to_csv(OUT_DIR / 'significant_pairs.csv', index=False)
if 'Attrition_flag' in corr.columns:
    corr[['Attrition_flag']].to_csv(OUT_DIR / 'corr_with_attrition.csv')

fig = plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(corr, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r', center=0, linewidths=.5, cbar_kws={"shrink": .8})
plt.title('Корреляционная матрица', fontsize=14, fontweight='bold')
save_fig(fig, '6_correlation_matrix.png')

if 'YearsAtCompany' in df.columns:
    df['YearsAtCompany_int'] = df['YearsAtCompany'].round().astype(int)
    years_attr = df.groupby('YearsAtCompany_int')['Attrition_flag'].agg(['mean', 'count'])
    years_attr = years_attr[years_attr['count'] > 5]
    fig = plt.figure(figsize=(10, 6))
    plt.plot(years_attr.index, years_attr['mean'], marker='o', color=COLOR_SCHEME['negative'])
    plt.fill_between(years_attr.index, years_attr['mean'], alpha=0.2, color=COLOR_SCHEME['negative'])
    plt.xlabel('Лет в компании')
    plt.ylabel('Доля текучести')
    plt.title('Доля текучести в зависимости от стажа', fontsize=14, fontweight='bold')
    save_fig(fig, '5_attrition_by_years.png')



numeric_features = df.select_dtypes(include=[np.number]).drop(columns=['Attrition_flag'], errors='ignore')
scaler = StandardScaler()
X_scaled = scaler.fit_transform(numeric_features)
fa = FactorAnalysis(n_components=5, random_state=42)
factors = fa.fit_transform(X_scaled)
fa_df = pd.DataFrame(factors, columns=[f'Factor_{i+1}' for i in range(factors.shape[1])])
fa_df['Attrition_flag'] = df['Attrition_flag'].values

X_train, X_test, y_train, y_test = train_test_split(fa_df.drop(columns='Attrition_flag'), fa_df['Attrition_flag'], test_size=0.3, random_state=42)
lr = LogisticRegression()
lr.fit(X_train, y_train)
odds_ratios = np.exp(lr.coef_[0])
or_df = pd.DataFrame({'factor': fa_df.drop(columns='Attrition_flag').columns, 'odds_ratio': odds_ratios})
or_df = or_df.sort_values('odds_ratio', ascending=False)

fa_cols = fa_df.columns[:-1]
if len(fa_cols) >= 3:
    top_factors = or_df.head(3).copy()
    top_factors['Фактор'] = ['Фактор 1: Профессиональный опыт и доход',
                             'Фактор 2: Удовлетворённость и баланс',
                             'Фактор 3: Возраст и расстояние от дома']
    fig = plt.figure(figsize=(9, 5))
    sns.barplot(x='odds_ratio', y='Фактор', data=top_factors,
                palette=[COLOR_SCHEME['accent'], COLOR_SCHEME['positive'], COLOR_SCHEME['primary']])
    plt.title('Влияние факторов на вероятность увольнения (Odds Ratio)', fontsize=14, fontweight='bold')
    plt.xlabel('Отношение шансов (Odds Ratio)')
    plt.ylabel('')
    plt.xlim(0, max(top_factors['odds_ratio'].max() + 0.5, 2.5))
    for i, v in enumerate(top_factors['odds_ratio']):
        plt.text(v + 0.05, i, f"{v:.2f}", va='center', fontsize=11)
    plt.grid(True, axis='x', alpha=0.3)
    plt.tight_layout()
    save_fig(fig, '7_factor_odds_rus.png')


Loaded: WA_Fn-UseC_-HR-Employee-Attrition.csv shape: (1470, 36)
Saved: hr_coursework_output/1_attrition_pie.png
Saved: hr_coursework_output/2_age_distribution.png
Saved: hr_coursework_output/3_income_boxplot.png
Saved: hr_coursework_output/4_attrition_by_jobrole.png
Saved: hr_coursework_output/6_correlation_matrix.png
Saved: hr_coursework_output/5_attrition_by_years.png
Saved: hr_coursework_output/7_factor_odds_rus.png
